In [3]:
import datetime 

import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
import folium 
import geopandas as gpd 
current_file_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_file_path, '..','..','..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from constants.paths import FOLDER_PATH
from load_inputs.systematic_analysis.load_netmob import load_dataset,get_list_apps,get_list_days,get_DL_UL_paths,read_csv,load_netmob_gdf
from load_inputs.systematic_analysis.utils import get_histogram_per_day_type,get_boxplot_per_spatial_unit_per_day_type,heatmap
from load_inputs.systematic_analysis.utils import IN_bdc,OUT_bdc,IN_nbdc,OUT_nbdc

folder_path = f'{FOLDER_PATH}/../raw_data/NetMob/'
shp_iris_path = f'{FOLDER_PATH}/../raw_data/lyon_iris_shapefile/'
path_save = '../../../../../../data/NetMob/NetMob_Lyon'
folder_path = '../../../../../../data/NetMob/NetMob_raw'

# Load Spatial Dataset

In [48]:
Netmob_gdf_joined,working_zones,n_rows,n_cols = load_netmob_gdf(folder_path,shp_iris_path)
Netmob_gdf_joined = Netmob_gdf_joined.drop_duplicates('tile_id')
Netmob_gdf_joined = Netmob_gdf_joined[['tile_id','INSEE_COM','NOM_COM','NOM_IRIS','geometry']]
Netmob_gdf_joined = gpd.GeoDataFrame(Netmob_gdf_joined,crs='EPSG:4326') #Don't know why, but the geodataframe seem corrupted as we can't convert it into GeoJson, that's why we need to use "gpd.GeoDataFrame()"
Netmob_gdf_joined
Netmob_gdf_joined.head()

,tile_id,INSEE_COM,NOM_COM,NOM_IRIS,geometry
5067,34807,69204.0,Saint-Genis-Laval,Basses Barolles,"POLYGON ((4.78896 45.66473, 4.78900 45.66563, ..."
5194,35094,69204.0,Saint-Genis-Laval,Basses Barolles,"POLYGON ((4.78900 45.66563, 4.78904 45.66653, ..."
5195,35095,69204.0,Saint-Genis-Laval,Basses Barolles,"POLYGON ((4.79028 45.66560, 4.79032 45.66650, ..."
5196,35096,69204.0,Saint-Genis-Laval,Basses Barolles,"POLYGON ((4.79156 45.66557, 4.79160 45.66647, ..."
5328,35380,69204.0,Saint-Genis-Laval,Basses Barolles,"POLYGON ((4.78776 45.66656, 4.78780 45.66745, ..."


# Load Raw data:

In [13]:

app_i = 2
day_i = 0

# apps = get_list_apps(folder_path)
# print(apps)

app = 'Google_Maps' 

""" Choices: 

['Apple_Video', 'Google_Play_Store', 'Google_Maps', 'Web_Clothes', 'Uber', 'Twitter', 'Microsoft_Mail', 'Microsoft_Store', 'Apple_Music', 'Microsoft_Office', 'Pokemon_GO', 'Clash_of_Clans', 'Yahoo_Mail', 'PlayStation', 'Wikipedia', 'Apple_Web_Services', 'Pinterest', 'Web_Ads', 'Google_Mail', 'Google_Meet', 'Apple_Siri', 'Web_Adult', 'Spotify', 'Deezer', 'Waze', 'Web_Games', 'Apple_App_Store', 'Microsoft_Skydrive', 'Google_Docs', 'Microsoft_Web_Services', 'Molotov', 'YouTube', 'Apple_iTunes', 'Apple_iMessage', 'DailyMotion', 'Netflix', 'Web_Transportation', 'Web_Downloads', 'SoundCloud', 'TeamViewer', 'Google_Web_Services', 'Facebook', 'EA_Games', 'Tor', 'Amazon_Web_Services', 'Web_e-Commerce', 'Telegram', 'Apple_Mail', 'Dropbox', 'Web_Food', 'Apple_iCloud', 'Skype', 'Facebook_Messenger', 'Twitch', 'Microsoft_Azure', 'Instagram', 'Facebook_Live', 'Web_Streaming', 'Orange_TV', 'Periscope', 'Snapchat', 'Web_Finance', 'WhatsApp', 'Web_Weather', 'Google_Drive', 'LinkedIn', 'Yahoo', 'Fortnite']
"""

# Get csv paths: 
list_days = get_list_days(folder_path,app)

df_tot = pd.DataFrame()
for k,day in enumerate(list_days):
    print(f"Processing day {k+1} / {len(list_days)}: {day}")
    year = day[:4]
    month = day[4:6]
    day_day = day[6:8]
    txt_paths = get_DL_UL_paths(folder_path,app,day)
    txt_path_DL = txt_paths[0]

    # Load df
    df = read_csv(txt_path_DL,day).T
    df.index = pd.to_datetime([f"{year}-{month}-{day_day} {t}" for t in df.index],format="%Y-%m-%d %H:%M")
    df_tot = pd.concat([df_tot,df])
    

# Restrict NetMob to Lyon: 
df = df_tot.sort_index()[list(Netmob_gdf_joined['tile_id'])]



# Save Dataset: 
np.save(f'{path_save}/{app}.npy', df.values)
pd.Series(df.index).to_pickle(f'{path_save}/index.pkl')
pd.Series(df.columns).to_pickle(f'{path_save}/columns.pkl')

Processing day 1 / 77: 20190504
Processing day 2 / 77: 20190503
Processing day 3 / 77: 20190330
Processing day 4 / 77: 20190502
Processing day 5 / 77: 20190505
Processing day 6 / 77: 20190331
Processing day 7 / 77: 20190415
Processing day 8 / 77: 20190412
Processing day 9 / 77: 20190424
Processing day 10 / 77: 20190423
Processing day 11 / 77: 20190422
Processing day 12 / 77: 20190425
Processing day 13 / 77: 20190413
Processing day 14 / 77: 20190414
Processing day 15 / 77: 20190409
Processing day 16 / 77: 20190407
Processing day 17 / 77: 20190401
Processing day 18 / 77: 20190406
Processing day 19 / 77: 20190430
Processing day 20 / 77: 20190408
Processing day 21 / 77: 20190322
Processing day 22 / 77: 20190325
Processing day 23 / 77: 20190527
Processing day 24 / 77: 20190518
Processing day 25 / 77: 20190520
Processing day 26 / 77: 20190516
Processing day 27 / 77: 20190529
Processing day 28 / 77: 20190511
Processing day 29 / 77: 20190324
Processing day 30 / 77: 20190323
Processing day 31 /

tile_id,66,353,354,640,641,642,643,926,927,928,...,121458,121459,121460,121461,121462,121744,121745,121746,121747,122032
2019-05-04 00:00:00,117.0,131.0,105.0,103.0,104.0,174.0,177.0,65.0,67.0,70.0,...,36.0,34.0,32.0,29.0,29.0,29.0,32.0,33.0,33.0,34.0
2019-05-04 00:15:00,120.0,133.0,109.0,101.0,107.0,155.0,155.0,60.0,62.0,71.0,...,16.0,15.0,13.0,12.0,12.0,13.0,13.0,12.0,12.0,13.0
2019-05-04 00:30:00,113.0,126.0,104.0,94.0,102.0,171.0,173.0,54.0,61.0,71.0,...,29.0,27.0,25.0,23.0,24.0,22.0,23.0,23.0,23.0,22.0
2019-05-04 00:45:00,98.0,112.0,90.0,87.0,91.0,149.0,146.0,48.0,55.0,62.0,...,19.0,18.0,16.0,15.0,15.0,15.0,14.0,14.0,14.0,13.0
2019-05-04 01:00:00,41.0,48.0,36.0,41.0,40.0,90.0,91.0,29.0,30.0,31.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


# Load saved data:

In [46]:
# Load Datasets:
data = np.load(f'{path_save}/{app}.npy')
idx = pd.read_pickle(f'{path_save}/index.pkl')
cols = pd.read_pickle(f'{path_save}/columns.pkl')

df_loaded = pd.DataFrame(data, index=idx, columns=cols)
df_loaded.shape

(7392, 27446)

## HeatMap:

In [82]:
import folium
from folium.plugins import HeatMap
from load_inputs.systematic_analysis.utils import preprocess_df

filter_q = 0.99 # None
period = 'evening_peak'   # 'all_day','morning_peak','evening_peak' # None
city = 'Lyon'
df = df_loaded.copy()

# filter quantile: 
if period is None: 
   df = preprocess_df(df,filter_q = filter_q)
else:
   df_bd_in, df_nbd_in = preprocess_df(df,period=period,city= city,filter_q = filter_q)
   df = pd.concat([df_bd_in,df_nbd_in])


ts = df.sum(0)

df_map = Netmob_gdf_joined.merge(ts.rename('volume'), on='tile_id').to_crs(epsg=4326)

Number of remaining time-slots after filtering 561
Number of remaining time-slots after filtering 286


In [83]:
centroids = df_map.geometry.centroid
heat_data = [[point.y, point.x, vol] for point, vol in zip(centroids, df_map['volume'])]
m = folium.Map(location=[centroids.y.mean(), centroids.x.mean()], zoom_start=11,tiles = 'CartoDB positron')
HeatMap(
    heat_data,
    radius=10,
    blur=20,
    min_opacity=0,
    gradient={0.3: 'blue', 0.5: 'lime', 0.7: 'yellow', 1: 'red'}
).add_to(m)
m

/tmp/ipykernel_150751/1589716492.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = df_map.geometry.centroid


## Load data aggregated by tags (ex: IRIS)


NetMob_selected_apps= ['Google_Maps'] 
NetMob_transfer_mode = ['DL'] 
NetMob_selected_tags = ['iris']
NetMob_expanded = ''
NetMob_only_epsilon = False
city = 'Lyon'
signal = 'netmob_POIs'
ds, df_raw, df = load_dataset(signal,
            NetMob_selected_apps,
            NetMob_transfer_mode,
            NetMob_selected_tags,
            NetMob_expanded,
            NetMob_only_epsilon,
            )
df_raw.columns.name = 'Spatial Unit ID'
df_raw.head()